In [ ]:
# This notebook creates a Multi-Agent solution to write User stories and System tests for a Functional Business Requirement

# CrewAI is used as the Multi-Agent framework 
# Three Agents - Business Analyst, System Tester, System Test Reviewer form the crew
# Three Tasks - Write User Stories, Write System Tests, Review System Tests are performed by each of these agents

# Claude Haiku (Anthropic) is used as the LLM by all these agents

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Install Crew AI, Tools, Langchain, Google Gemini
!pip install crewai crewai_tools langchain_community langchain_anthropic --quiet
print("---")
!pip show crewai crewai_tools langchain_community langchain_anthropic

In [ ]:
# Necessary imports
import os
from getpass import getpass
from crewai import Agent, Task, Crew
from langchain_anthropic import ChatAnthropic

In [ ]:
# Get API Key for Google Gemini
os.environ["ANTHROPIC_API_KEY"] = getpass("Enter ANTHROPIC_API_KEY: ")

llm_claude = ChatAnthropic(model='claude-3-haiku-20240307', temperature=0.8)

In [ ]:
# Create our First Agent
# Role = Business Analyst
# Responsibility = Take a business requirement and come up with a set of Agile user stories 

business_analyst = Agent(
    role="Business Analyst",
    goal="Write an accurate and complete set of agile user stories for the {requirement}",
    backstory="You're working on writing agile user stories "
              "for the requirement: {requirement}. "
              "You capture details in the user story that helps the "
              "tester to ensure the software is built right "
              "and complete in all aspects. "
              "Your work is the basis for "
              "the Tester to write system test cases for this requirement.",
    allow_delegation=False,
	verbose=True,
    llm = llm_claude
)

In [ ]:
# Create our Second Agent
# Role = System Tester
# Responsibility = Use the user stories created by the Business Analyst, come up with system test cases to test the user story implementation

system_tester = Agent(
    role="System Tester",
    goal="Write system test cases that will test all functionalities that are implemented through user stories, "
         "which implements the requirement: {requirement}",
    backstory="You're working on creating "
              "system test cases for the requirement: {requirement}. "
              "You base your test cases on the work of "
              "the Business Analyst, who provides the user stories "
              "and relevant information about the requirement. "
              "You follow the main features and "
              "direction of the user story, "
              "as provided by the Content Planner. "
              "You must also ensure to cover boundary conditions and error scenarios "
              "that are relevant to complete test the user stories "
              "provide by the Business Analyst. "
              "You must ensure that the system test cases "
              "provide maxiumum coverage "
              "to test all aspects of the requirement.",
    allow_delegation=False,
    verbose=True,
    llm = llm_claude
)

In [ ]:
# Creating our Final Agent
# Role = System Test Reviewer
# Responsibility = Review the system tests created by the System Tester, improve test coverage, scenarios etc.

system_test_reviewer = Agent(
    role="System Test Reviewer",
    goal="Review a given set of System Test Cases to align with "
         "the standards of the organization and coverage to the user stories and requirement: {requirement} ",
    backstory="You are an system test reviewer who receives a set of system test cases "
              "from the System Tester. "
              "Your goal is to review the system test cases "
              "to ensure that it follows industry best practices, organization standards "
              "provides balanced review "
              "when providing improvement or suggestions, "
              "and also ensures highest cover based on the "
              "user stories that are written by the Business Analyst.",
    allow_delegation=False,
    verbose=True,
    llm = llm_claude
)

In [ ]:
# Define the task that will be assigned to the Business Analyst

write_stories = Task(
    description=(
        "1. Understand the functional business requirement "
            "and relevant context on the requirement {requirement}.\n"
        "2. Identify the users or business stakeholders involved in the requirement considering "
            "their interests and outcomes relevant to the requirement.\n"
        "3. Develop a set of agile user stories that wil help to "
            "implement the requirement as a software product.\n"
        "4. Include assumptions, acceptance criteria, error conditions as part of the generated user stories."
    ),
    expected_output="A comprehensive set of agile user stories using As [a user persona], I want [to perform this action] so that [I can accomplish this goal] format "
        "with description, outcomes, acceptance criteria, "
        "assumptions, and error conditions.",
    agent=business_analyst,
)

In [ ]:
# Define the task that will be assigned to the Content Writer

write_tests = Task(
    description=(
        "1. Use the user stories created by Business Analyst to create a set of "
            "System test cases for the requirement {requirement}.\n"
        "2. Ensure coverage for all scenarios described in the user stories.\n"
		"3. Error conditions and edge cases must be captured "
            "to ensure completeness with the tests.\n"
        "4. Ensure the test cases are structured as "
            "test case id, test description, pre-conditions, test steps, expected results, "
            "test data, actual results and test status.  Leave actual results and test status empty, to be filled during execution\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written set of system test cases "
        "that covers all functionalities described in the user stories, "
        "with a well-defined test description, pre-conditions, test steps, expected results, test data, actual results and test status.",
    agent=system_tester,
)

In [ ]:
# Define the task that will be assigned to the Content Editor

review_tests = Task(
    description=("Review the system test cases generated by System Tester for "
                 "completeness, coverage against user stories and "
                 "conformance to the requirement {requirement}"),
    expected_output="A well-written and reviewed system test cases in markdown format, "
                    "ready for test execution. ",
    agent=system_test_reviewer
)

In [ ]:
# Bring all agents and tasks together into a Crew

crew = Crew(
    agents=[business_analyst, system_tester, system_test_reviewer],
    tasks=[write_stories, write_tests, review_tests],
    verbose=2
)

In [ ]:
# Run the tasks in sequence for the topic provided here

result = crew.kickoff(inputs={"requirement": "The system must allow blog visitors to sign up for the newsletter by leaving their email."})

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
requirement = "The system must allow users to verify their accounts using their phone number"
result = crew.kickoff(inputs={"requirement": requirement})

In [ ]:
Markdown(result)